# 05 — Fusión Híbrida y Evaluación
Se combinan puntajes de CF y Contenido, y se reportan métricas Top‑N.

In [ ]:
from src.eval.metrics import recall_at_k, ndcg_at_k

truth = {5, 8, 13, 21}
recs = [21, 9, 1, 5, 33, 8, 6, 13, 2, 3]
print('Recall@5:', recall_at_k(truth, recs, k=5))
print('NDCG@10:', ndcg_at_k(truth, recs, k=10))
